In [1]:
from datasets import load_dataset
t_dataset = load_dataset("csv", data_files='data4_3/train_data.csv', delimiter=",")
te_dataset = load_dataset("csv", data_files='data4_3/test_data.csv', delimiter=",")
t_dataset['test'] = te_dataset['train']
t_dataset

C:\Users\PPTI Java\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\PPTI Java\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Extracting data files: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.23it/s]
Generating train split: 170 examples [00:00, 8343.85 examples/s]
Extracting data files: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 987.59it/s]
Generating train split: 74 examples [00:00, 21044.04 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 170
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 74
    })
})

In [2]:
from transformers import AutoTokenizer

task='sentiment'
# MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
MODEL = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)


tokenized_datasets = t_dataset.map(tokenize_function, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 3613.38 examples/s]


In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import numpy as np
# import evaluate

# metric = evaluate.load("accuracy")

from datasets import load_metric
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")

C:\Users\PPTI Java\AppData\Local\Temp\ipykernel_28220\215590796.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")


In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    
    return {"accuracy": accuracy, "f1": f1, "precision": precision}



In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    adam_beta1=0.9,
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    learning_rate=25e-6, 
    num_train_epochs=10.0
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()
model.save_pretrained("models/model_3")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision
1,No log,0.265666,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
2,No log,0.248387,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
3,No log,0.303157,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
4,No log,0.345469,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
5,No log,0.352026,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
6,No log,0.348393,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
7,No log,0.398331,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
8,No log,0.384449,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
9,No log,0.355780,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}
10,No log,0.360381,{'accuracy': 0.9324324324324325},{'f1': 0.8998298998298998},{'precision': 0.8694302410518627}


C:\Users\PPTI Java\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'accuracy': 0.9324324324324325}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8998298998298998}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8694302410518627}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropp